### LLM Applications using LCEL(LangChain Expression Language)

### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [12]:
## using openAI and Groq AI

import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')



In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model='llama-3.1-8b-instant', groq_api_key=groq_api_key)
# model = ChatGroq(model='openai/gpt-oss-120b', groq_api_key=groq_api_key)

model

c:\Coding\AI_ML_DL\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000243FF1B2D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000243FF1B2C80>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content='Translate the following from English to French'),
    HumanMessage(content='Hello, How are you?')
]

model.invoke(messages)

AIMessage(content='Bonjour, comment allez-vous ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 48, 'total_tokens': 56, 'completion_time': 0.01907485, 'completion_tokens_details': None, 'prompt_time': 0.003673395, 'prompt_tokens_details': None, 'queue_time': 0.056766495, 'total_time': 0.022748245}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b90d3-7dd4-7411-8e4e-e570faca8aa2-0', usage_metadata={'input_tokens': 48, 'output_tokens': 8, 'total_tokens': 56})

In [4]:

messages = [
    SystemMessage(content='Translate the following from English to Hindi'),
    HumanMessage(content='Hello, How are you?')
]

model.invoke(messages)

AIMessage(content='नमस्ते, मैं ठीक हूँ।', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 48, 'total_tokens': 61, 'completion_time': 0.023110272, 'completion_tokens_details': None, 'prompt_time': 0.06581711, 'prompt_tokens_details': None, 'queue_time': 0.11520434, 'total_time': 0.088927382}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b90d3-7ec8-7913-84be-92d759434547-0', usage_metadata={'input_tokens': 48, 'output_tokens': 13, 'total_tokens': 61})

In [5]:
response = model.invoke(messages)
response

AIMessage(content='नमस्ते, मैं ठीक हूँ, धन्यवाद।', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 48, 'total_tokens': 65, 'completion_time': 0.02823876, 'completion_tokens_details': None, 'prompt_time': 0.003422197, 'prompt_tokens_details': None, 'queue_time': 0.051428542, 'total_time': 0.031660957}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b90d3-7fe0-7ab2-b276-c3ae6c1598b6-0', usage_metadata={'input_tokens': 48, 'output_tokens': 17, 'total_tokens': 65})

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(response)

'नमस्ते, मैं ठीक हूँ, धन्यवाद।'

In [7]:
## using LCEL - chain the component

chain = model | parser

chain.invoke(messages)

'नमस्ते, आप कैसे हैं?'

In [8]:
## Prompt template

from langchain_core.prompts import ChatPromptTemplate
generic_prompt = "Translate the following into {language}"

prompt = ChatPromptTemplate.from_messages([('system', generic_prompt), ('user', '{text}')])

In [9]:
response = prompt.invoke({'language':'Hindi', 'text':'Hello'})

In [11]:
response.to_messages()

[SystemMessage(content='Translate the following into Hindi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [10]:
chain = prompt | model | parser

chain.invoke({'language':'Hindi', 'text':'Hello'})

'नमस्ते'